<a href="https://colab.research.google.com/github/yousrabougara/WalmartSalesForecasting/blob/main/CatBoost_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
i!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.preprocessing import StandardScaler ,PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression , Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from xgboost import XGBRegressor
from category_encoders import BinaryEncoder
import pickle
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('Walmart.csv')

In [ ]:
df.head(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [ ]:
df.shape

(6435, 8)

In [ ]:
df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [ ]:
df['WeekOfMonth']=pd.to_datetime(df['Date'], format='%d-%m-%Y').dt.isocalendar().week
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,WeekOfMonth
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106,5
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106,6
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106,7
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106,8
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106,9


In [ ]:
X=df.drop(['Date', 'Weekly_Sales'], axis=1)
Y=df['Weekly_Sales']
X.head().to_string()

'   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  WeekOfMonth\n0      1             0        42.31       2.572  211.096358         8.106            5\n1      1             1        38.51       2.548  211.242170         8.106            6\n2      1             0        39.93       2.514  211.289143         8.106            7\n3      1             0        46.63       2.561  211.319643         8.106            8\n4      1             0        46.50       2.625  211.350143         8.106            9'

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X, Y.values.reshape(-1, 1))
Y = np.log(Y)
Y

,Weekly_Sales
0,14.312455
1,14.311400
2,14.292966
3,14.158907
4,14.256862
...,...
6430,13.477481
6431,13.505522
6432,13.506897
6433,13.484400


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=6, eval_metric='MAE', verbose=10, random_state=42)
model = model.fit(X_train, y_train, eval_set=(X_test, y_test))
model

0:	learn: 0.4846694	test: 0.4952746	best: 0.4952746 (0)	total: 57.3ms	remaining: 57.2s
10:	learn: 0.4045917	test: 0.4115368	best: 0.4115368 (10)	total: 184ms	remaining: 16.5s
20:	learn: 0.3512969	test: 0.3586090	best: 0.3586090 (20)	total: 269ms	remaining: 12.5s
30:	learn: 0.3134412	test: 0.3192029	best: 0.3192029 (30)	total: 348ms	remaining: 10.9s
40:	learn: 0.2794642	test: 0.2839663	best: 0.2839663 (40)	total: 421ms	remaining: 9.85s
50:	learn: 0.2527873	test: 0.2569166	best: 0.2569166 (50)	total: 498ms	remaining: 9.27s
60:	learn: 0.2326658	test: 0.2373751	best: 0.2373751 (60)	total: 562ms	remaining: 8.66s
70:	learn: 0.2130611	test: 0.2179232	best: 0.2179232 (70)	total: 647ms	remaining: 8.46s
80:	learn: 0.1999961	test: 0.2042596	best: 0.2042596 (80)	total: 749ms	remaining: 8.5s
90:	learn: 0.1878741	test: 0.1917536	best: 0.1917536 (90)	total: 810ms	remaining: 8.09s
100:	learn: 0.1746255	test: 0.1786614	best: 0.1786614 (100)	total: 877ms	remaining: 7.81s
110:	learn: 0.1643402	test: 0.16

In [ ]:
#Prediction
prediction = model.predict(X_test)

In [ ]:
#R2
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_test, prediction)
print("R-squared (R2) score:", r2)

R-squared (R2) score: 0.9893649628123325


In [ ]:
# Afficher les précisions
print("Training set Accuracy:", round(model.score(X_train, y_train), 3) * 100)
print("Test set Accuracy:", round(model.score(X_test, y_test), 3) * 100)

Training set Accuracy: 99.3
Test set Accuracy: 98.9


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split



def objective(trial):
    # Définir les hyperparamètres spécifiques à CatBoost
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.1, log=True),
        "depth": trial.suggest_int("depth", 4, 7),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 1.0),
    }

    # Créer le modèle avec les paramètres proposés
    model = CatBoostRegressor(
        **params,
        verbose=0,  # Désactiver les logs pendant l'entraînement
        random_seed=42,
    )

    # Entraîner le modèle
    model.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = model.predict(X_test)

    # Calculer le score R² (le score à maximiser)
    r2 = r2_score(y_test, y_pred)

    return r2



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Créer une étude Optuna pour maximiser le score R²
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Afficher les meilleurs paramètres trouvés par Optuna
best_params = study.best_params
print("Best parameters found by Optuna:", best_params)

# Construire le modèle avec les meilleurs paramètres trouvés
catboost_best = CatBoostRegressor(
    **best_params,
    verbose=0,  # Désactiver les logs
    random_seed=42,
)

# Entraîner le modèle final sur le jeu d'entraînement
catboost_best.fit(X_train, y_train)

# Évaluation sur le jeu de test
y_pred = catboost_best.predict(X_test)

# Calculer les scores d'évaluation
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

# Afficher les résultats
print("Final Test R2 Score:", r2)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)

# Afficher les précisions
print("Training set Accuracy:", round(catboost_best.score(X_train, y_train), 3) * 100)
print("Test set Accuracy:", round(catboost_best.score(X_test, y_test), 3) * 100)


[I 2025-01-21 17:44:07,756] A new study created in memory with name: no-name-f14346d0-664d-4ce3-b067-6b55c7ab3547
[I 2025-01-21 17:44:10,638] Trial 0 finished with value: 0.8848902823061028 and parameters: {'iterations': 732, 'learning_rate': 0.013644381375314023, 'depth': 4, 'l2_leaf_reg': 4.0879070076380035, 'bagging_temperature': 0.9246754422729935, 'random_strength': 0.6353185359327463}. Best is trial 0 with value: 0.8848902823061028.
[I 2025-01-21 17:44:12,732] Trial 1 finished with value: 0.9873096085422984 and parameters: {'iterations': 636, 'learning_rate': 0.0943475908861697, 'depth': 4, 'l2_leaf_reg': 3.84519215393836, 'bagging_temperature': 0.5902518522050275, 'random_strength': 0.025778132840683976}. Best is trial 1 with value: 0.9873096085422984.
[I 2025-01-21 17:44:13,208] Trial 2 finished with value: 0.7786842426039704 and parameters: {'iterations': 116, 'learning_rate': 0.046057266514366285, 'depth': 4, 'l2_leaf_reg': 3.5486495053123894, 'bagging_temperature': 0.8475000

Best parameters found by Optuna: {'iterations': 879, 'learning_rate': 0.08906974418912089, 'depth': 7, 'l2_leaf_reg': 3.753692336164798, 'bagging_temperature': 0.8745740514122993, 'random_strength': 0.20716110169280547}
Final Test R2 Score: 0.9915706688938567
Mean Squared Error (MSE): 0.003010379490927806
Root Mean Squared Error (RMSE): 0.05486692529136115
Mean Absolute Error (MAE): 0.038765008927555515
Training set Accuracy: 99.7
Test set Accuracy: 99.2
